In [1]:
##用于可视化
%matplotlib inline  

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets('/home/liuzixuan/机器学习/深度/小练习/MNIST_data',one_hot=True,reshape=False)

Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/liuzixuan/机器学习/深度/小练习/MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
#二维卷积时输入是4阶张量，一维卷积输入是3阶张量
#1代表通道数
mnist.train.images.shape

(55000, 28, 28, 1)

In [14]:
with tf.Graph().as_default() as g:
    inputs = tf.placeholder(shape=[None,28,28,1],dtype=tf.float32)
    labels = tf.placeholder(shape=[None,10],dtype=tf.float32)
    
    #定义模型
    #第一组：
    #8个5*5的卷积核
    conv1_weight = tf.get_variable('c1_w',[5,5,1,8],dtype=tf.float32)
    conv1_bias = tf.get_variable('c1_b',[8],dtype=tf.float32)
    
    conv1 = tf.nn.conv2d(inputs,conv1_weight,[1,1,1,1],'VALID')#[1,1,1,1]为步长
    conv1 = tf.nn.relu(conv1 + conv1_bias)#输出shape=[None,24,24,8]
    
    mean1,var1 = tf.nn.moments(conv1,axes=0)
    ema1 = tf.train.ExponentialMovingAverage(decay=0.99)
    ema_apply1 = ema1.apply([mean1,var1])
    output1 = tf.nn.batch_normalization(conv1,ema1.average(mean1),ema1.average(var1),offset=0,scale=1,variance_epsilon=0.001)
    
    #输出shape = [None,12,12,8]
    pool1 = tf.nn.max_pool(output1,[1,2,2,1],[1,2,2,1],'SAME')#第一个[1,2,2,1]为滑窗大小，第二个为步长大小，池化时滑窗与步长一般一样
    
    #第二组：
    conv2_weight = tf.get_variable('c2_w',[5,5,8,16],dtype=tf.float32)
    conv2_bias = tf.get_variable('c2_d',[16],dtype=tf.float32)
    
    conv2 = tf.nn.conv2d(pool1,conv2_weight,[1,1,1,1],'VALID')#步长可以是[2,2,2,2]，可以省略之后池化的步骤
    conv2 = tf.nn.relu(conv2 + conv2_bias)#shape=[None,8,8,16]
    
    mean２,var２ = tf.nn.moments(conv２,axes=0)
    ema２ = tf.train.ExponentialMovingAverage(decay=0.99)
    ema_apply２ = ema２.apply([mean２,var２])
    output２ = tf.nn.batch_normalization(conv２,ema２.average(mean２),ema２.average(var２),offset=0,scale=1,variance_epsilon=0.001)
    
    pool2 = tf.nn.max_pool(output2,[1,2,2,1],[1,2,2,1],'SAME')#shape=[4,4,16]
    
    #第三组：
    conv3_weight = tf.get_variable('c3_w',[3,3,16,16],dtype=tf.float32)
    conv3_bias = tf.get_variable('c3_d',[16],dtype=tf.float32)
    
    conv3 = tf.nn.conv2d(pool2,conv3_weight,[1,1,1,1],'VALID')#步长可以是[2,2,2,2]，可以省略之后池化的步骤
    conv3 = tf.nn.relu(conv3 + conv3_bias)#shape=[None,2,2,16]
    
    #mean3,var3 = tf.nn.moments(conv3,axes=0)
    #ema3 = tf.train.ExponentialMovingAverage(decay=0.99)
    #ema_apply3 = ema２.apply([mean3,var3])
    #output3 = tf.nn.batch_normalization(conv3,ema3.average(mean3),ema3.average(var3),offset=0,scale=1,variance_epsilon=0.001)
    
    pool3 = tf.nn.max_pool(conv3,[1,2,2,1],[1,2,2,1],'SAME')#shape=[1,1,16]
    
    tmp = tf.reshape(pool3,[-1,1 * 1 * 16])
    #tmp = tf.reshape(pool2,[-1,pool2_shape[1]*pool2_shape[2]*pool2_shape[3]])
    #输出层为全连接层
    logits = tf.keras.layers.Dense(10,activation=None)(tmp)
    output = tf.nn.softmax(logits)
    
    #loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(labels,output))
    loss = tf.reduce_mean(-tf.reduce_sum(labels * tf.log(output + 1e-17),axis=1))
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(output,axis=1),tf.argmax(labels,axis=1)),tf.float32))
    
    ##优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    ##最小化代价函数
    train_op = optim.minimize(loss)

In [16]:
##训练
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(30000):
        batch_images,batch_labels = mnist.train.next_batch(32)
        res_loss,_ = sess.run([loss,train_op],feed_dict={inputs:batch_images,labels:batch_labels})
        if i%500==0:
            accs = []
            for j in range(10000//32):
                batch_images,batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc,feed_dict={inputs:batch_images,labels:batch_labels})
                accs.append(res_acc)
            m_acc = np.mean(accs)
            print('loss %2.4f,acc %2.2f%%' % (res_loss,m_acc * 100))
            

loss 34.2510,acc 8.61%
loss 35.4742,acc 9.96%
loss 34.2510,acc 10.55%
loss 35.4742,acc 10.32%
loss 35.4742,acc 10.28%
loss 36.6974,acc 10.62%
loss 34.2510,acc 9.91%
loss 37.9207,acc 10.37%
loss 36.6974,acc 10.42%
loss 35.4742,acc 10.43%
loss 31.8045,acc 10.18%
loss 35.4742,acc 10.30%
loss 37.9207,acc 10.42%
loss 34.2510,acc 10.25%
loss 33.0277,acc 10.20%
loss 36.6974,acc 10.50%
loss 33.0277,acc 10.14%
loss 36.6974,acc 10.51%
loss 37.9207,acc 10.40%
loss 33.0277,acc 10.10%
loss 36.6974,acc 10.54%
loss 36.6974,acc 10.00%
loss 33.0277,acc 10.53%
loss 36.6974,acc 10.34%
loss 34.2510,acc 10.09%
loss 34.2510,acc 10.48%
loss 39.1439,acc 10.44%
loss 36.6974,acc 10.22%
loss 34.2510,acc 10.50%
loss 36.6974,acc 9.99%
loss 31.8045,acc 10.50%
loss 37.9207,acc 10.36%
loss 36.6974,acc 10.53%
loss 34.2510,acc 10.20%
loss 31.8045,acc 10.12%
loss 35.4742,acc 10.39%
loss 37.9207,acc 10.43%
loss 34.2510,acc 10.36%
loss 34.2510,acc 10.22%
loss 36.6974,acc 10.28%
loss 36.6974,acc 10.35%
loss 34.2510,acc 10.